In [1]:
from source.regency import Regency
from source.run import rungame
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%matplotlib notebook

In [3]:
X = 40
my_game = rungame('TestGame', train=True, GameLength=X, IntDC = 15)
Game = my_game.Game


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Continuing Game TestGame at Season 0 Action 3


In [8]:
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)
Game.take_domain_actions()
Game.take_domain_actions()
Game.take_domain_actions()
Game.War

error 31 Mhiction 1 - Initiative 17


KeyboardInterrupt: 

In [5]:
Game.War

,Year,Location,Event,Notes
0,"Taelinir, year 1524 HC",Bhalaene,Battle of Bhalaene,Battle of Bhalaene\n\nBelligerents:\nGhorien H...
0,"Taelinir, year 1524 HC",Haesrien,Battle of Haesrien,Battle of Haesrien\n\nBelligerents:\nCarilon A...


In [13]:
import numpy as np
import pandas as pd
self = Game

Regent, decision, actor, Type, state, capital, high_pop, low_pop, friend, enemy, rando, enemy_capital = Game.errors[0]

In [14]:
state[3], state[94], (decision[30], state[99]), (decision[31], state[100])

(0, 0, (0.0, 0), (1.0, 1))

In [17]:
hType = 'Source'
N = 37
Target = None

In [44]:

temp = pd.concat([self.Provences[self.Provences['Regent']==Regent][['Provence']].copy()
              ,self.Holdings[self.Holdings['Regent']==Regent][['Provence']].copy()], sort=False).drop_duplicates()
# all neighboring provences to regent
temp = pd.merge(temp, self.Geography.copy(), on='Provence', how='left').drop_duplicates()
temp = temp[temp['Border']==1]
temp['Provence'] = temp['Neighbor']

temp_ = pd.merge(temp[['Provence']], self.Holdings[self.Holdings['Type'] == Type].copy(), on='Provence', how='left')
dct = {}
dct['Provence'] = []
for p in set(temp_['Provence']):
    temp__ = temp_[temp_['Regent']==Regent]
    if temp__[temp__['Provence']==p].shape[0]==0:
        dct['Provence'].append(p)
temp_check = pd.DataFrame(dct)
# # Validity

temp_check = pd.merge(temp_check, self.Provences[['Provence', 'Population', 'Regent']].copy(), on='Provence', how='left')
temp_check = pd.merge(temp_check, self.Relationships[self.Relationships['Regent'] ==Regent][['Other', 'Diplomacy']], left_on='Regent', right_on='Other', how='left').fillna(0)

temp_check['Where'] = temp_check['Population'] - temp_check['Diplomacy']*decision[28]
temp_check = pd.merge(temp_check[['Provence', 'Where', 'Population']], self.Holdings[self.Holdings['Type']==Type].copy(), on='Provence', how='left')
temp_check = pd.merge(temp_check, self.Relationships[self.Relationships['Regent'] ==Regent][['Other', 'Diplomacy']], left_on='Regent', right_on='Other', how='left').fillna(0)

temp_check['Where'] = temp_check['Where'] - temp_check['Diplomacy']
temp_check = temp_check.sort_values('Where', ascending=False)
df = self.Holdings[self.Holdings['Regent']==Regent][self.Holdings['Type']==hType].copy()
df['Check'] = df['Type']
temp_check = pd.merge(temp_check, df[['Provence','Check']], on='Provence', how='left').fillna(0)
# temp_check = temp_check[temp_check['Check']==0]
# # More likely to set up shop in rival area
if temp_check.shape[0]>0:
    Target = temp_check.iloc[0]['Provence']
    
    
Target

'Serien'